## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [get `job_source_uri`](#get-job_source_uri)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
primary_key = os.getenv('PK')
swan_orchestrator = swan.resource(service_name='Orchestrator')

### Show available hardware information

In [2]:
hardwares_info = swan_orchestrator.get_hardware_config()
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'descripti

### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [3]:
result = swan_orchestrator.create_task(
    image_name="hello-world",
    # hardware_id=hardware_id,
    wallet_address=wallet_address,
    primary_key=primary_key,
    # region='North Carolina-US',
    # duration=duration
    # auto_pay=False,
    # image_name="hello world"
)
print(json.dumps(result, indent=2))
task_uuid = result['id']

{'tx_hash': '0xc4e4f87a95d55401fb0032feb33af2d98851ba799be0d99734ddb04b6cc7f3eb', 'task_uuid': '8eab3244-4f9d-4947-8db1-0bbf6a98579d'}
{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719524874,
      "end_at": 1719528474,
      "id": 4410,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719524874,
      "start_in": 300,
      "status": "created",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719524874,
        "duration": 3600,
        "end_at": 1719528474,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/QmVpRohoDXLis1QLZrZKrYMD5dyo4Z4rP8G59F8aKFSsKY",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "r

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [4]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "beneficiary": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "cp_account_address": "0xd63853b23Af4cc3b37377368dd33F09b8567aCea",
        "created_at": 1718386123,
        "id": 71,
        "lat": 35.8639,
        "lon": -78.535,
        "multi_address": [
          "/ip4/provider.cp.filezoo.com.cn/tcp/9087"
        ],
        "node_id": "0475e6cb8909e125db8b233e9d2c8237e28e8d852fb4dcf1d5f9b5eead94931d8ed84c5f08e4e88a53b24d80bb3bb3b181389fcddf0114ac93f2866d769cc8ff18",
        "online": 1,
        "owner_address": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "region": "North Carolina-US",
        "task_types": "[3]",
        "updated_at": 1719524881,
        "version": "2.0",
        "worker_address": "0x1AE8665873565C9089753920A0866C49Bd35DCC9"
      },
      {
        "beneficiary": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "cp_account_address": "0x4D81F4277667862046802bDdcdB667704bEFbbAB",
       

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [5]:
r = swan_orchestrator.get_real_url(task_uuid)
print(r)

['https://ifn55bbpx2.cp.filezoo.com.cn', 'https://9tl0erm1k9.dev2.crosschain.computer']


In [6]:
duration = 3600
renew_task = swan_orchestrator.renew_task(task_uuid=task_uuid, 
                                 duration=duration * 2,
                                 primary_key=primary_key
                                 # hardware_id=hardware_id
                                )
print(renew_task)

{'data': {'job': [{'build_log': 'wss://log.cp.filezoo.com.cn:9087/api/v1/computing/lagrange/spaces/log?space_id=QmVpRohoDXLis1QLZrZKrYMD5dyo4Z4rP8G59F8aKFSsKY&type=build', 'comments': 'buildImage: updated job_result_uri..', 'container_log': 'wss://log.cp.filezoo.com.cn:9087/api/v1/computing/lagrange/spaces/log?space_id=QmVpRohoDXLis1QLZrZKrYMD5dyo4Z4rP8G59F8aKFSsKY&type=container', 'cp_account_address': '0xd63853b23Af4cc3b37377368dd33F09b8567aCea', 'created_at': 1719524883, 'duration': 10800, 'ended_at': None, 'hardware': 'C1ae.small', 'id': 4497, 'job_real_uri': 'https://ifn55bbpx2.cp.filezoo.com.cn', 'job_result_uri': 'https://42f6d9f62851.acl.swanipfs.com/ipfs/QmZQqPMG5Ho921vDk3DmXaUJ4ETPmKpGY3XM831xideGAz', 'job_source_uri': 'https://plutotest.acl.swanipfs.com/ipfs/QmVpRohoDXLis1QLZrZKrYMD5dyo4Z4rP8G59F8aKFSsKY', 'name': 'Job-406ecd2b-4572-40a7-ba28-de67eb14242b', 'node_id': '0475e6cb8909e125db8b233e9d2c8237e28e8d852fb4dcf1d5f9b5eead94931d8ed84c5f08e4e88a53b24d80bb3bb3b181389fcddf0

In [9]:
r = swan_orchestrator.terminate_task(task_uuid=task_uuid)
print(r)

{'data': {'retryable': False, 'task_status': 'terminate_pending'}, 'message': 'Task termination is queued', 'status': 'success'}


In [8]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


Expecting value: line 1 column 1 (char 0)
<Response [200]>
